In [20]:
% pylab inline

Populating the interactive namespace from numpy and matplotlib


/home/cscorley/envs/triage/lib/python2.7/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['f']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [21]:
from __future__ import print_function
import os.path
import pandas
import src
import sklearn
import os
import scipy
import scipy.stats
import csv

In [22]:
def fake(*args, **kwargs):
    print('Fake called with', str(args), str(kwargs))
    sys.exit(1)

# fake out the create_model so we don't accidentally attempt to create data
src.common.create_model = fake

In [23]:
print(os.getcwd())
if os.getcwd().endswith('notebooks'):
    os.chdir('..')
print(os.getcwd())

/home/cscorley/git/triage
/home/cscorley/git/triage


In [24]:
args = dict(level='file', force=False, model='lda', source=['release', 'changeset', 'temporal'], random_seed_value=1)

model_config, model_config_string = src.main.get_default_model_config(args)
args.update({'model_config': model_config, 'model_config_string': model_config_string})

changeset_config, changeset_config_string = src.main.get_default_changeset_config()
args.update({'changeset_config': changeset_config, 'changeset_config_string': changeset_config_string})

projects = src.common.load_projects(args)
projects

[Project(name='tika', printable_name='Tika v1.8', version='v1.8', ref='refs/tags/1.8', data_path='data/tika/', full_path='data/tika/v1.8/', src_path='data/tika/v1.8/src/', changeset_config_string=u'True-True-False-True', source=['release', 'changeset', 'temporal'], model_config_string=u'seed1-batch-0.002-0.5-0.002-1000-1000-500-1.0-1', force=False, level='file', model_config={'passes': 1, 'eta': 0.002, 'num_topics': 500, 'iterations': 1000, 'decay': 0.5, 'algorithm': 'batch', 'alpha': 0.002, 'offset': 1.0, 'max_bound_iterations': 1000}, model='lda', random_seed_value=1, changeset_config={'include_removals': True, 'include_context': True, 'include_message': False, 'include_additions': True}),
 Project(name='pig', printable_name='Pig v0.14.0', version='v0.14.0', ref='refs/tags/release-0.14.0', data_path='data/pig/', full_path='data/pig/v0.14.0/', src_path='data/pig/v0.14.0/src/', changeset_config_string=u'True-True-False-True', source=['release', 'changeset', 'temporal'], model_config_st

In [30]:
for project in projects:
    ids = src.common.load_ids(project)
    issue2git, git2issue = src.common.load_issue2git(project, ids, filter_ids=True)
    assert len(issue2git.keys()) == len(ids), "%d vs %d" % (len(issue2git.keys()), len(ids))
    goldset = src.goldsets.load_goldset(project)
    
    with open(os.path.join(project.full_path, 'changes-file-goldset.csv'), 'w') as f:
        w =  csv.writer(f)
        w.writerow(['sha', 'issues', 'change_type', 'name'])
        for sha, changes in goldset.items():
            commit, change_list = changes
            for change_type, name in change_list:
                if sha in git2issue:
                    issues = set(git2issue[sha])
                    w.writerow([sha, ';'.join(issues), change_type, name])
                    
for project in projects:
    ids = src.common.load_ids(project)
    issue2git, git2issue = src.common.load_issue2git(project, ids)
    assert len(issue2git.keys()) == len(ids), "%d vs %d" % (len(issue2git.keys()), len(ids))
    goldset = src.goldsets.load_goldset(project)
    
    with open(os.path.join(project.full_path, 'changes-file-goldset.csv'), 'w') as f:
        w =  csv.writer(f)
        w.writerow(['sha', 'issues', 'change_type', 'name'])
        for sha, changes in goldset.items():
            commit, change_list = changes
            for change_type, name in change_list:
                if sha in git2issue:
                    issues = set(git2issue[sha])
                    w.writerow([sha, ';'.join(issues), change_type, name])

In [26]:
for project in projects:
    goldset = src.goldsets.load_goldset(project)
    
    with open(os.path.join(project.data_path, 'changes-file-full.csv'), 'w') as f:
        w =  csv.writer(f)
        w.writerow(['sha', 'change_type', 'name'])
        for sha, changes in goldset.items():
            commit, change_list = changes
            for change_type, name in change_list:
                w.writerow([sha, change_type, name])